<a href="https://colab.research.google.com/github/natureLanguageQing/BERT-BiLSTM-CRF-NER/blob/master/clis_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/bojone/lic2020_baselines.git

Cloning into 'lic2020_baselines'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 39 (delta 12), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (39/39), done.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!wget http://bj.bcebos.com/v1/ai-studio-online/e521ab850c7046669d576e2c0c2794666c47c622db1d4484bbbe3e7bed5da8ae?responseContentDisposition=attachment%3B%20filename%3Dsample5.json.zip&authorization=bce-auth-v1%2F0ef6765c1e494918bc0d4c3ca3e5c6d1%2F2020-03-09T10%3A33%3A23Z%2F-1%2F%2F197fcd92c227979f99526c0027081fe27d17d6c0fad3577afdaac63ba7960a4d

--2020-03-31 13:13:31--  http://bj.bcebos.com/v1/ai-studio-online/e521ab850c7046669d576e2c0c2794666c47c622db1d4484bbbe3e7bed5da8ae?responseContentDisposition=attachment%3B%20filename%3Dsample5.json.zip
Resolving bj.bcebos.com (bj.bcebos.com)... 103.235.46.61
Connecting to bj.bcebos.com (bj.bcebos.com)|103.235.46.61|:80... connected.
HTTP request sent, awaiting response... 403 Forbidden
2020-03-31 13:13:33 ERROR 403: Forbidden.



In [0]:
!nvidia-smi

Wed Apr  1 12:14:51 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!unzip train_data.json

Archive:  train_data.json.zip
   creating: train_data/
  inflating: train_data/train.json   
  inflating: train_data/License.docx  
   creating: __MACOSX/train_data/
  inflating: __MACOSX/train_data/._License.docx  


In [0]:
!unzip dev_data.json.zip

Archive:  dev_data.json.zip
   creating: dev_data/
  inflating: dev_data/dev.json       
  inflating: dev_data/License.docx   
   creating: __MACOSX/dev_data/
  inflating: __MACOSX/dev_data/._License.docx  


In [0]:
!unzip /content/event_schema.zip

Archive:  /content/event_schema.zip
   creating: event_schema/
  inflating: event_schema/.DS_Store  
   creating: __MACOSX/event_schema/
  inflating: __MACOSX/event_schema/._.DS_Store  
  inflating: event_schema/License.docx  
  inflating: __MACOSX/event_schema/._License.docx  
  inflating: event_schema/event_schema.pdf  
  inflating: __MACOSX/event_schema/._event_schema.pdf  
  inflating: event_schema/event_schema.json  
  inflating: __MACOSX/event_schema/._event_schema.json  


In [0]:
!cp -i /content/drive/My\ Drive/electra_tiny.zip（副本） file2


In [0]:
!pip install bert4keras

  Created wheel for bert4keras: filename=bert4keras-0.6.9-cp36-none-any.whl size=33997 sha256=abf5785ca20bfd1a5edf6c7b53ad1505e82748834afe9dc6ad8b50212afcd5eb
  Stored in directory: /root/.cache/pip/wheels/96/8b/04/0d1b06be2562f3e80a83cc6fd86257ed45e568a90293a9ab20
Successfully built bert4keras


In [0]:
!unzip file2

Archive:  file2
  inflating: bert_config_tiny.json   
   creating: __MACOSX/
  inflating: __MACOSX/._bert_config_tiny.json  
  inflating: model.ckpt-1000000.data-00000-of-00001  
  inflating: __MACOSX/._model.ckpt-1000000.data-00000-of-00001  
  inflating: model.ckpt-1000000.index  
  inflating: __MACOSX/._model.ckpt-1000000.index  
  inflating: model.ckpt-1000000.meta  
  inflating: __MACOSX/._model.ckpt-1000000.meta  
  inflating: vocab.txt               
  inflating: __MACOSX/._vocab.txt    


In [0]:
#! -*- coding: utf-8 -*-
# 百度LIC2020的事件抽取赛道，非官方baseline
# 直接用BERT+CRF
# 在第一期测试集上能达到0.73的F1，略优于官方baseline

import json
import numpy as np
from bert4keras.backend import keras, K
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer
from bert4keras.optimizers import Adam
from bert4keras.snippets import sequence_padding, DataGenerator
from bert4keras.snippets import open
from bert4keras.layers import ConditionalRandomField
from keras.layers import Dense
from keras.models import Model
from tqdm import tqdm
import pylcs

# 基本信息
maxlen = 128
epochs = 20
batch_size = 32
learning_rate = 1e-5
crf_lr_multiplier = 1000  # 必要时扩大CRF层的学习率

# bert配置
config_path = 'bert_config_tiny.json'
checkpoint_path = 'model.ckpt-1000000'
dict_path = 'vocab.txt'


def load_data(filename):
    D = []
    with open(filename) as f:
        for l in f:
            l = json.loads(l)
            arguments = {}
            for event in l['event_list']:
                for argument in event['arguments']:
                    key = argument['argument']
                    value = (event['event_type'], argument['role'])
                    arguments[key] = value
            D.append((l['text'], arguments))
    return D


# 读取数据
train_data = load_data('train_data/train.json')
valid_data = load_data('dev_data/dev.json')

# 读取schema
with open('event_schema/event_schema.json') as f:
    id2label, label2id, n = {}, {}, 0
    for l in f:
        l = json.loads(l)
        for role in l['role_list']:
            key = (l['event_type'], role['role'])
            id2label[n] = key
            label2id[key] = n
            n += 1
    num_labels = len(id2label) * 2 + 1

# 建立分词器
tokenizer = Tokenizer(dict_path, do_lower_case=True)


def search(pattern, sequence):
    """从sequence中寻找子串pattern
    如果找到，返回第一个下标；否则返回-1。
    """
    n = len(pattern)
    for i in range(len(sequence)):
        if sequence[i:i + n] == pattern:
            return i
    return -1


class data_generator(DataGenerator):
    """数据生成器
    """
    def __iter__(self, random=False):
        batch_token_ids, batch_segment_ids, batch_labels = [], [], []
        for is_end, (text, arguments) in self.sample(random):
            token_ids, segment_ids = tokenizer.encode(text, max_length=maxlen)
            labels = [0] * len(token_ids)
            for argument in arguments.items():
                a_token_ids = tokenizer.encode(argument[0])[0][1:-1]
                start_index = search(a_token_ids, token_ids)
                if start_index != -1:
                    labels[start_index] = label2id[argument[1]] * 2 + 1
                    for i in range(1, len(a_token_ids)):
                        labels[start_index + i] = label2id[argument[1]] * 2 + 2
            batch_token_ids.append(token_ids)
            batch_segment_ids.append(segment_ids)
            batch_labels.append(labels)
            if len(batch_token_ids) == self.batch_size or is_end:
                batch_token_ids = sequence_padding(batch_token_ids)
                batch_segment_ids = sequence_padding(batch_segment_ids)
                batch_labels = sequence_padding(batch_labels)
                yield [batch_token_ids, batch_segment_ids], batch_labels
                batch_token_ids, batch_segment_ids, batch_labels = [], [], []


model = build_transformer_model(
    config_path,
    checkpoint_path,
    model = 'electra'
)

output = Dense(num_labels)(model.output)
CRF = ConditionalRandomField(lr_multiplier=crf_lr_multiplier)
output = CRF(output)

model = Model(model.input, output)
model.summary()

model.compile(
    loss=CRF.sparse_loss,
    optimizer=Adam(learning_rate),
    metrics=[CRF.sparse_accuracy]
)


def viterbi_decode(nodes, trans):
    """Viterbi算法求最优路径
    其中nodes.shape=[seq_len, num_labels],
        trans.shape=[num_labels, num_labels].
    """
    labels = np.arange(num_labels).reshape((1, -1))
    scores = nodes[0].reshape((-1, 1))
    scores[1:] -= np.inf  # 第一个标签必然是0
    paths = labels
    for l in range(1, len(nodes)):
        M = scores + trans + nodes[l].reshape((1, -1))
        idxs = M.argmax(0)
        scores = M.max(0).reshape((-1, 1))
        paths = np.concatenate([paths[:, idxs], labels], 0)
    return paths[:, scores[0].argmax()]


def extract_arguments(text):
    """命名实体识别函数
    """
    tokens = tokenizer.tokenize(text)
    while len(tokens) > 512:
        tokens.pop(-2)
    token_ids = tokenizer.tokens_to_ids(tokens)
    segment_ids = [0] * len(token_ids)
    nodes = model.predict([[token_ids], [segment_ids]])[0]
    trans = K.eval(CRF.trans)
    labels = viterbi_decode(nodes, trans)[1:-1]
    arguments, starting = [], False
    for token, label in zip(tokens[1:-1], labels):
        if label > 0:
            if label % 2 == 1:
                starting = True
                arguments.append([[token], id2label[(label - 1) // 2]])
            elif starting:
                arguments[-1][0].append(token)
            else:
                starting = False
        else:
            starting = False

    return {tokenizer.decode(w, w): l for w, l in arguments}


def evaluate(data):
    """评测函数（跟官方评测结果不一定相同，但很接近）
    """
    X, Y, Z = 1e-10, 1e-10, 1e-10
    for text, arguments in tqdm(data):
        inv_arguments = {v: k for k, v in arguments.items()}
        pred_arguments = extract_arguments(text)
        pred_inv_arguments = {v: k for k, v in pred_arguments.items()}
        Y += len(pred_inv_arguments)
        Z += len(inv_arguments)
        for k, v in pred_inv_arguments.items():
            if k in inv_arguments:
                # 用最长公共子串作为匹配程度度量
                l = pylcs.lcs(v, inv_arguments[k])
                X += 2. * l / (len(v) + len(inv_arguments[k]))
    f1, precision, recall = 2 * X / (Y + Z), X / Y, X / Z
    return f1, precision, recall


def predict_to_file(in_file, out_file):
    """预测结果到文件，方便提交
    """
    fw = open(out_file, 'w', encoding='utf-8')
    with open(in_file) as fr:
        for l in tqdm(fr):
            l = json.loads(l)
            arguments = extract_arguments(l['text'])
            event_list = []
            for k, v in arguments.items():
                event_list.append({
                    'event_type': v[0],
                    'arguments': [{
                        'role': v[1],
                        'argument': k
                    }]
                })
            l['event_list'] = event_list
            l = json.dumps(l, ensure_ascii=False)
            fw.write(l + '\n')
    fw.close()


class Evaluator(keras.callbacks.Callback):
    """评估和保存模型
    """
    def __init__(self):
        self.best_val_f1 = 0.

    def on_epoch_end(self, epoch, logs=None):
        f1, precision, recall = evaluate(valid_data)
        if f1 >= self.best_val_f1:
            self.best_val_f1 = f1
            model.save_weights('best_model.weights')
        print(
            'f1: %.5f, precision: %.5f, recall: %.5f, best f1: %.5f\n' %
            (f1, precision, recall, self.best_val_f1)
        )


if __name__ == '__main__':

    train_generator = data_generator(train_data, batch_size)
    evaluator = Evaluator()

    model.fit_generator(
        train_generator.forfit(),
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        callbacks=[evaluator]
    )

else:

    model.load_weights('best_model.weights')
    # predict_to_file('/root/baidu/datasets/ee/test1_data/test1.json', 'ee_pred.json')


In [0]:
!pip install pylcs

     |████████████████████████████████| 296kB 3.5MB/s 
  ERROR: Failed building wheel for pylcs
  Running setup.py clean for pylcs
Failed to build pylcs
    Running setup.py install for pylcs ... done


In [0]:
!pip uninstall -y tensorflow


In [0]:
!pip install tensorflow-gpu==1.14.0


     |████████████████████████████████| 377.0MB 44kB/s 
     |████████████████████████████████| 491kB 60.9MB/s 
     |████████████████████████████████| 3.2MB 29.7MB/s 
ERROR: tensorflow 2.2.0rc2 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.14.0 which is incompatible.
ERROR: tensorflow 2.2.0rc2 has requirement tensorflow-estimator<2.3.0,>=2.2.0rc0, but you'll have tensorflow-estimator 1.14.0 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0


In [0]:
!unzip /content/event_schema.zip

unzip:  cannot find or open /content/event_schema.zip, /content/event_schema.zip.zip or /content/event_schema.zip.ZIP.


In [0]:
!pip install pylcs

     |████████████████████████████████| 296kB 2.8MB/s 
  ERROR: Failed building wheel for pylcs
  Running setup.py clean for pylcs
Failed to build pylcs
    Running setup.py install for pylcs ... done


In [0]:
import os
import os
import kashgari
from kashgari.embeddings import BERTEmbeddingV2
from kashgari.tokenizer import BertTokenizer
import pandas as pd
import keras
import numpy as np
kashgari.config.use_cudnn_cell = True
model_folder = 'electra_tiny'

checkpoint_path = os.path.join('model.ckpt-1000000')
config_path = os.path.join('bert_config_tiny.json')
vocab_path = os.path.join('vocab.txt')

def data_doubles(train_data):
    train_features = [list(i[0]) for i in train_data]
    train_labels = [i[1] for i in train_data]
    return train_features, train_labels



def read_message(path):
    data = pd.read_csv(path).values.tolist()
    random_order = list(range(len(data)))
    np.random.shuffle(random_order)
    train_data, valid_data, test_data = [], [], []
    for i, j in enumerate(random_order):
        if i % 10 != 0 and i % 10 != 1:
            train_data.append(data[i])
        if i % 10 == 0:
            valid_data.append(data[i])
        if i % 10 == 1:
            test_data.append(data[i])
    train_features, train_labels = data_doubles(train_data)
    valid_features, valid_labels = data_doubles(valid_data)
    test_features, test_labels = data_doubles(test_data)
    return train_features, train_labels, valid_features, valid_labels, test_features, test_labels
train_features, train_labels, valid_features, valid_labels, test_features, test_labels = read_message("five_CLS_data.csv")

feature_list = []
label_list = []
max_len = 0
# 根据数据集中的最大信息长度判断当前任务的max-len
for i in five_CLS_data:
    feature_list.append(i[0])
    if len(i[0]) > max_len:
        max_len = len(i[0])
    label_list.append(i[1])
tokenizer = BertTokenizer.load_from_vacab_file(vocab_path)
embed = BERTEmbeddingV2(vocab_path,
                        config_path,
                        checkpoint_path,
                        bert_type='electra',
                        task=kashgari.CLASSIFICATION,
                        sequence_length=max_len)

tf_board_callback = keras.callbacks.TensorBoard(log_dir='tf_dir', update_freq=1000)



from kashgari.tasks.classification import CNNLSTMModel

model = CNNLSTMModel(embed)

# ------------ build model ------------
model.fit(
    train_features, train_labels,
    valid_features, valid_labels,
    epochs=30,
    batch_size=256,
    callbacks =[tf_board_callback]
)
model.evaluate(test_features, test_labels)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: DeprecationWarning: The 'load_from_vacab_file' function is deprecated, use 'load_from_vocab_file' instead
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:22: UserWarning: bert4keras.bert has been renamed as bert4keras.models.
  warnings.warn('bert4keras.bert has been renamed as bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:23: UserWarning: please use bert4keras.models.
  warnings.warn('please use bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:44: UserWarning: bert4keras.tokenizer has been renamed as bert4keras.tokenizers.
  warnings.warn('bert4keras.tokenizer has been renamed as bert4keras.tokenizers.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:45: UserWarning: please use bert4keras.tokenizers.
  warnings.warn('please use bert4keras.tokenizers.')


Model: "model_23"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, None)]       0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     (None, None, 312)    6591936     Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, None, 312)    624         Input-Segment[0][0]              
___________________________________________________________________________________________

In [0]:
!wget https://drive.google.com/uc?export=download&confirm=t4ju&id=1UP4byt4-kgenwST0KvyMYNbln6FfaSLp

--2020-04-01 02:33:15--  https://drive.google.com/uc?export=download
Resolving drive.google.com (drive.google.com)... 172.217.204.100, 172.217.204.139, 172.217.204.138, ...
Connecting to drive.google.com (drive.google.com)|172.217.204.100|:443... connected.
HTTP request sent, awaiting response... 400 Bad Request
2020-04-01 02:33:15 ERROR 400: Bad Request.



In [0]:
!wget https://drive.google.com/uc?export=download&confirm=t4ju&id=1UP4byt4-kgenwST0KvyMYNbln6FfaSLp

ERROR! Session/line number was not unique in database. History logging moved to new session 60
--2020-04-01 02:36:32--  https://drive.google.com/uc?export=download
Resolving drive.google.com (drive.google.com)... 108.177.125.139, 108.177.125.101, 108.177.125.138, ...
Connecting to drive.google.com (drive.google.com)|108.177.125.139|:443... connected.
HTTP request sent, awaiting response... 400 Bad Request
2020-04-01 02:36:32 ERROR 400: Bad Request.



In [0]:
%tensorflow_version 1.x

TensorFlow is already loaded. Please restart the runtime to change versions.


In [0]:
!unzip /content/drive/My\ Drive/electra_tiny.zip（副本）

Archive:  /content/drive/My Drive/electra_tiny.zip（副本）
  inflating: bert_config_tiny.json   
   creating: __MACOSX/
  inflating: __MACOSX/._bert_config_tiny.json  
  inflating: model.ckpt-1000000.data-00000-of-00001  
  inflating: __MACOSX/._model.ckpt-1000000.data-00000-of-00001  
  inflating: model.ckpt-1000000.index  
  inflating: __MACOSX/._model.ckpt-1000000.index  
  inflating: model.ckpt-1000000.meta  
  inflating: __MACOSX/._model.ckpt-1000000.meta  
  inflating: vocab.txt               
  inflating: __MACOSX/._vocab.txt    


In [0]:
!pip uninstall -y tensorflow


Uninstalling tensorflow-2.2.0rc2:
  Successfully uninstalled tensorflow-2.2.0rc2


In [0]:
!pip install bert4keras

  Created wheel for bert4keras: filename=bert4keras-0.6.9-cp36-none-any.whl size=33997 sha256=32feb00f7a1e191ffd5d9a42b87fb6614aab305a73bc5740f8b6a16036ccfc09
  Stored in directory: /root/.cache/pip/wheels/96/8b/04/0d1b06be2562f3e80a83cc6fd86257ed45e568a90293a9ab20
Successfully built bert4keras


In [0]:
!pip install tensorflow-gpu==1.14.0

     |████████████████████████████████| 377.0MB 43kB/s 
     |████████████████████████████████| 3.2MB 23.2MB/s 
     |████████████████████████████████| 491kB 42.3MB/s 
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0


In [0]:
!nvidia-smi

In [0]:
!pip uninstall kashgari && pip install git+https://github.com/BrikerMan/Kashgari


  Cloning https://github.com/BrikerMan/Kashgari to /tmp/pip-req-build-o7yxmpew
  Running command git clone -q https://github.com/BrikerMan/Kashgari /tmp/pip-req-build-o7yxmpew
     |████████████████████████████████| 17.3MB 2.5MB/s 
  Created wheel for kashgari: filename=kashgari-1.1.4-cp36-none-any.whl size=88070 sha256=68233fe8908843e7a4df61712a8434f9f78b84811fcfccb0ed016ac559ed7760
  Stored in directory: /tmp/pip-ephem-wheel-cache-fdcjdi2l/wheels/b0/42/7d/7e096aa7f533a5ccee6a31cd62bde62a6bde90116a864a9132
  Created wheel for keras-bert: filename=keras_bert-0.81.0-cp36-none-any.whl size=37913 sha256=5c0b4e76c42eb6391dafeb31acc28ad6bed29775924a409e240b9d3aaab0800d
  Stored in directory: /root/.cache/pip/wheels/bd/27/da/ffc2d573aa48b87440ec4f98bc7c992e3a2d899edb2d22ef9e
  Created wheel for keras-gpt-2: filename=keras_gpt_2-0.14.0-cp36-none-any.whl size=10525 sha256=e75bb69e45b2879c78854b073d5656211a1f960e09923edd7897c67eafdc19fa
  Stored in directory: /root/.cache/pip/wheels/ec/d8/06/ba

In [0]:
!unzip /content/drive/My\ Drive/electra_tiny.zip

Archive:  /content/drive/My Drive/electra_tiny.zip
  inflating: bert_config_tiny.json   
  inflating: __MACOSX/._bert_config_tiny.json  
  inflating: model.ckpt-1000000.data-00000-of-00001  
  inflating: __MACOSX/._model.ckpt-1000000.data-00000-of-00001  
  inflating: model.ckpt-1000000.index  
  inflating: __MACOSX/._model.ckpt-1000000.index  
  inflating: model.ckpt-1000000.meta  
  inflating: __MACOSX/._model.ckpt-1000000.meta  
  inflating: vocab.txt               
  inflating: __MACOSX/._vocab.txt    
